### 파이썬 라이브러리 불러오기


In [3]:
# Data Import
import pydsptools.biorad as biorad

# Config
import pydsptools.config.pda as pda

# DSP processing
import pydsp.run.worker
import os
import pathlib as Path

# Analysis
import polars as pl
import pandas as pd

# Visualization
import pydsptools.plot as dspplt

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Script Description

## [2024-04-23]
    
* 배경
    * 이 스크립트는 데이터의 수와 종류가 늘어나고 병합과정이 복잡해짐에 따라 각 데이터가 각 단계에서 올바르게 처리되는지를 알아본다.
    * baseline fitting 알고리즘의 성능을 측정하는 방법 중 하나인 MuDT의 왜곡 신호를 비교하는 시각화를 수행하기 위해 데이터 엔지니어링이 적절히 수행됐는지 확인해야한다.
    * MuDT 왜곡 신호란? 하나의 well에서 High Temp에서 양성 Low Temp에서 음성 신호 결과가 나왔을 때, MuDT의 신호 왜곡 현상인 과도한 CR 현상이 발생하는 것을 의미한다.
    * 각 각의 well에서 High Temp에서 양성 Low Temp에서 음성 신호를 filtering 해야하는데 1개의 well당 Low Temp 신호와 High Temp 신호의 pairing이 전제가 되어야 한다.
* 목표
    * `cfx batch analyzer`에서 추출된 신호의 sample size를 확인하고 최종 data 전처리 결과물과 일치하는지 확인한다.
* 방법
    * 아래의 각 데이터 항목을 아래의 검증 절차에 따라 수행한다.
        * [After BPN] RFU: original rfu를 BPN으로 Transformation한 Data
        * [Modified DSP] Original ABSD: 음성탈락 신호들의 absd-orig을 얻기위해 설정값들을 변경하여 얻은 Data로 음성 데이터에서 **final_ct가 부정확**할 수 있다. (final_ct 부정확 with 설정값 modified)
        * [Auto] Baseline-Subtracted RFU: pgr-manager의 raw tab 결과 (final_ct 부정확 with 설정값 modified)
        * [CFX] Baseline-Subtracted RFU: cfx manager의 baseline subtracted data. (final_ct 부정확 with 설정값 modified)
        * [Strep] Baseline-Subtracted RFU (final_ct 부정확 with 설정값 modified)
        * [Strep+n] Baseline-Subtracted RFU (final_ct 부정확 with 설정값 modified)
        * [Control DSP] Baseline-Subtracted RFU (final_ct 정확 with 설정값 intact)
    * 검증 절차 
        * `1.cfx-to-parquet-converter.ipynb`의 csv에서 parquet의 pcrd 개수, pcrd당 channel당 temperature당 well 개수 확인
        * `2.dsp-executer.ipynb`의 DSP 실행 후 연산 결과인 parquet의 pcrd 개수, pcrd당 channel당 temperature당 well 개수 확인

## 1. CFX의 csv-to-parquet converter 단계

**1. CSV well 카운트**

In [19]:
csv_path = './data/GI-B-I/GI-B-I-100/csv'
GIBI_8strip_csv_df = []

channel_mapping = {
    'Cal Red 610': 'Cal Red 610',
    'FAM': 'FAM',
    'HEX': 'HEX',
    'Quasar 670': 'Quasar 670'
}
temperature_mapping = {
    'Amp4': 'Low',
    'Amp5': 'High'
}

def extract_channel(filename):
    for key in channel_mapping:
        if key in filename:
            return channel_mapping[key]
    return 'unexpected'

def extract_temperature(filename):
    for key in temperature_mapping:
        if key in filename:
            return temperature_mapping[key]
    return 'unexpected'

for filename in os.listdir(csv_path):
    if 'Quantification Amplification Results' in filename and filename.endswith('.csv'):
        file_path = os.path.join(csv_path, filename)
        temp = pd.read_csv(file_path)
        temp = temp.iloc[:,1:]
        temp['name'] = filename
        temp['channel'] = extract_channel(filename)
        temp['temperature'] = extract_temperature(filename)
        GIBI_8strip_csv_df.append(temp)

GIBI_8strip_csv_df = pd.concat(GIBI_8strip_csv_df, ignore_index=True)

# 칼럼 재정렬
leading_columns = ['name', 'channel', 'temperature']
other_columns = [col for col in GIBI_8strip_csv_df.columns if col not in leading_columns]
new_order = leading_columns + other_columns
GIBI_8strip_csv_df = GIBI_8strip_csv_df[new_order]

In [17]:
leading_columns = ['name', 'channel', 'temperature']
leading_columns.append('temp')

aggregated_df = pd.DataFrame()
for well in other_columns[1:]:
    current_columns = leading_columns[:-1] + ['Cycle', well]  # Simplified combining lists
    temp = combined_df[current_columns].groupby(['name', 'channel', 'temperature']).size().reset_index(name='count')
    temp['well'] = well
    aggregated_df = pd.concat([aggregated_df, temp], ignore_index=True)


In [29]:
well_counts = aggregated_df.groupby(['name', 'channel'])['well'].nunique().reset_index(name='unique_wells')
print(well_counts)
print(well_counts['unique_wells'].unique())

                                                  name      channel  \
0    2. admin_2017-05-22 14-06-05_BR101459 GI9801XY...  Cal Red 610   
1    2. admin_2017-05-22 14-06-05_BR101459 GI9801XY...          FAM   
2    2. admin_2017-05-22 14-06-05_BR101459 GI9801XY...          HEX   
3    2. admin_2017-05-22 14-06-05_BR101459 GI9801XY...   Quasar 670   
4    2. admin_2017-05-22 14-06-05_BR101459 GI9801XY...  Cal Red 610   
..                                                 ...          ...   
795  admin_2017-10-27 10-11-31_BR100858 GI9801Y 완제품...   Quasar 670   
796  admin_2017-10-27 10-11-31_BR100858 GI9801Y 완제품...  Cal Red 610   
797  admin_2017-10-27 10-11-31_BR100858 GI9801Y 완제품...          FAM   
798  admin_2017-10-27 10-11-31_BR100858 GI9801Y 완제품...          HEX   
799  admin_2017-10-27 10-11-31_BR100858 GI9801Y 완제품...   Quasar 670   

     unique_wells  
0              96  
1              96  
2              96  
3              96  
4              96  
..            ...  
795    

In [16]:
aggregated_df.groupby(['channel','temperature','count']).size().reset_index(name='n').assign(n=lambda df: df['n'] / 96)
# cycle 45개 protocol : 94개
# cycle 50개 protocol : 6개

,channel,temperature,count,n
0,Cal Red 610,High,45,94.0
1,Cal Red 610,High,50,6.0
2,Cal Red 610,Low,45,94.0
3,Cal Red 610,Low,50,6.0
4,FAM,High,45,94.0
5,FAM,High,50,6.0
6,FAM,Low,45,94.0
7,FAM,Low,50,6.0
8,HEX,High,45,94.0
9,HEX,High,50,6.0


**2. pda-pro의 DSP input의 well 카운트**

In [41]:
paths

<generator object Path.glob at 0x75449c7ff560>

In [42]:
parquet_path = './data/GI-B-I/GI-B-I-100/computed/pcr_results/'
paths = Path.Path(f"{parquet_path}").glob("*.parquet")
paths = sorted(list(paths), key=lambda p: p.name)
GIBI_8strip_parquet_df = pd.DataFrame()
for path in paths:
    name = path.name[:-8]
    df = pd.read_parquet(path)
    print({'unique_wells':len(df['well'].unique()),'unique_signals': len(df['well'])/4/2,'name':name}) #모든 pcrd파일들 channel 별로 high, low 짝지어 찍혔음

{'unique_wells': 84, 'unique_signals': 84.0, 'name': '2. admin_2017-05-22 14-06-05_BR101459 GI9801XY MOM 소량 반제품1 민감도(Vpara, CdB, Vchol)'}
{'unique_wells': 84, 'unique_signals': 84.0, 'name': '4. admin_2017-05-22 14-38-11_BR101644 GI9801XY MOM 소량 반제품1 민감도(Ahyd, Avero, Sbong)'}
{'unique_wells': 56, 'unique_signals': 56.0, 'name': '5. admin_2017-05-22 14-40-33_BR101645 GI9801XY MOM 소량 반제품1 민감도(Styphi, IC)'}
{'unique_wells': 96, 'unique_signals': 96.0, 'name': 'admin_2015-02-23 02-32-41_BR100160_Allplex GI_B1_TCF_specificity-1'}
{'unique_wells': 96, 'unique_signals': 96.0, 'name': 'admin_2015-02-23 14-34-49_BR100135_Allplex GI_B1_TCF_specificity-2'}
{'unique_wells': 96, 'unique_signals': 96.0, 'name': 'admin_2015-02-23 14-35-01_BR100172_Allplex GI_B1_TCF_specificity-3'}
{'unique_wells': 96, 'unique_signals': 96.0, 'name': 'admin_2015-02-24 10-10-02_BR100135_Allplex GI_B1_TCF_specificity-1'}
{'unique_wells': 96, 'unique_signals': 96.0, 'name': 'admin_2015-02-24 10-10-34_BR100159_Allplex GI_

### [After BPN] + [Modified DSP]

### [Control DSP]

In [43]:
parquet_path = './data/GI-B-I/control/computed/config__dsp2_generic/dsp/'
paths = Path.Path(f"{parquet_path}").glob("*.parquet")
paths = sorted(list(paths), key=lambda p: p.name)
GIBI_8strip_control_df = pd.DataFrame()
for path in paths:
    name = path.name[:-8]
    df = pd.read_parquet(path)
    print({'unique_wells':len(df['well'].unique()),'unique_signals': len(df['well'])/4/2,'name':name}) #모든 pcrd파일들 channel 별로 high, low 짝지어 찍혔음

{'unique_wells': 84, 'unique_signals': 84.0, 'name': '2. admin_2017-05-22 14-06-05_BR101459 GI9801XY MOM 소량 반제품1 민감도(Vpara, CdB, Vchol).dsp'}
{'unique_wells': 84, 'unique_signals': 84.0, 'name': '4. admin_2017-05-22 14-38-11_BR101644 GI9801XY MOM 소량 반제품1 민감도(Ahyd, Avero, Sbong).dsp'}
{'unique_wells': 56, 'unique_signals': 56.0, 'name': '5. admin_2017-05-22 14-40-33_BR101645 GI9801XY MOM 소량 반제품1 민감도(Styphi, IC).dsp'}
{'unique_wells': 96, 'unique_signals': 96.0, 'name': 'admin_2015-02-23 02-32-41_BR100160_Allplex GI_B1_TCF_specificity-1.dsp'}
{'unique_wells': 96, 'unique_signals': 96.0, 'name': 'admin_2015-02-23 14-34-49_BR100135_Allplex GI_B1_TCF_specificity-2.dsp'}
{'unique_wells': 96, 'unique_signals': 96.0, 'name': 'admin_2015-02-23 14-35-01_BR100172_Allplex GI_B1_TCF_specificity-3.dsp'}
{'unique_wells': 96, 'unique_signals': 96.0, 'name': 'admin_2015-02-24 10-10-02_BR100135_Allplex GI_B1_TCF_specificity-1.dsp'}
{'unique_wells': 96, 'unique_signals': 96.0, 'name': 'admin_2015-02-24 1

### [Auto]

### [CFX] 

### [Strep+1] 

## 2. dsp-executer 단계

### [After BPN] + [Modified DSP]

### [Control DSP]

### [Auto]

### [CFX] 

### [Strep+1] 